In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Read from silver layer
silver_df = spark.table("stock_market.silver_stock_data")

print(f"Silver records: {silver_df.count()}")
silver_df.orderBy("date").show(5)

In [0]:
# Define window for moving averages (ordered by date for each symbol)
window_7 = Window.partitionBy("symbol").orderBy("date").rowsBetween(-6, 0)
window_30 = Window.partitionBy("symbol").orderBy("date").rowsBetween(-29, 0)

# Calculate moving averages
gold_df = silver_df \
    .withColumn("ma_7_day", round(avg("close").over(window_7), 2)) \
    .withColumn("ma_30_day", round(avg("close").over(window_30), 2)) \
    .withColumn("volatility_7_day", round(stddev("close").over(window_7), 2))

# Show recent data with moving averages
gold_df.orderBy(col("date").desc()) \
    .select("date", "symbol", "close", "ma_7_day", "ma_30_day", "volatility_7_day") \
    .show(10)

In [0]:
# Add more business metrics
final_gold_df = gold_df \
    .withColumn("price_vs_ma7", round(col("close") - col("ma_7_day"), 2)) \
    .withColumn("price_vs_ma30", round(col("close") - col("ma_30_day"), 2)) \
    .withColumn("trend_signal", 
        when((col("close") > col("ma_7_day")) & (col("ma_7_day") > col("ma_30_day")), "Strong Bullish")
        .when(col("close") > col("ma_7_day"), "Bullish")
        .when(col("close") < col("ma_7_day"), "Bearish")
        .otherwise("Neutral"))

# Write to gold table
final_gold_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("stock_market.gold_stock_analytics")

print(f"Successfully wrote {final_gold_df.count()} records to gold layer")

# Show final analytics
spark.table("stock_market.gold_stock_analytics") \
    .orderBy(col("date").desc()) \
    .select("date", "close", "ma_7_day", "ma_30_day", "volatility_7_day", "trend_signal") \
    .show(5)